<a href="https://colab.research.google.com/github/adalves-ufabc/2021.QS-PLN/blob/main/2021_Q1_PLN_Notebook_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2021.Q1]**
Prof. Alexandre Donizeti Alves

### **Chatbot com Python e NLTK para pesquisar documentos**

### **01. Importação das bibliotecas**

In [1]:
import bs4 as bs
# pip install bs4
import urllib.request
import re
import nltk
import numpy as np
import random
import string

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install spacy --upgrade

     |████████████████████████████████| 12.8MB 246kB/s 
     |████████████████████████████████| 9.1MB 47.9MB/s 
     |████████████████████████████████| 1.1MB 52.2MB/s 
     |████████████████████████████████| 460kB 48.8MB/s 
     |████████████████████████████████| 122kB 52.3MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=98f905198625af13d3d1cf795f68eab0d245ddc68bb0a95d3b467308e8a4e01e
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: smart-open 4.2.0
    Uninstalling smart-open-4.2.0:
      Successfully uninstalled smart-open-4.2.0
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    U

In [4]:
import spacy
spacy.__version__
#pip install spacy==2.2.3

'3.0.5'

In [5]:
nltk.__version__

'3.2.5'

In [6]:
!python3 -m spacy download pt

2021-03-29 18:05:53.156891: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
⚠ As of spaCy v3.0, shortcuts like 'pt' are deprecated. Please use the
full pipeline package name 'pt_core_news_sm' instead.
     |████████████████████████████████| 22.1MB 1.4MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


### **02. Carregamento e pré-processamento da base de dados**

**Carregamento da base de dados**

In [8]:
dados = urllib.request.urlopen('https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial')

In [9]:
dados = dados.read()
dados

b'<!DOCTYPE html>\n<html class="client-nojs" lang="pt" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Intelig\xc3\xaancia artificial \xe2\x80\x93 Wikip\xc3\xa9dia, a enciclop\xc3\xa9dia livre</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\\t.","\xc2\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janeiro","fevereiro","mar\xc3\xa7o","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro"],"wgRequestId":"7ce0e187-410a-42e6-b47e-1c99eeef6a27","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Intelig\xc3\xaancia_artificial","wgTitle":"Intelig\xc3\xaancia artificial","wgCurRevisionId":60658094,"wgRevisionId":60658094,"wgArticleId":12722,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["!CS1 manut: Nomes m\xc3\xbaltiplos: lista

In [10]:
dados_html = bs.BeautifulSoup(dados, 'lxml')
#dados_html

In [11]:
paragrafos = dados_html.find_all('p')

In [12]:
len(paragrafos)

65

In [13]:
paragrafos[0]

<p><b>Inteligência artificial</b> (por vezes mencionada pela sigla em português <b>IA</b> ou pela sigla em inglês <b>AI</b> - <i>artificial intelligence</i>) é a <a href="/wiki/Intelig%C3%AAncia" title="Inteligência">inteligência</a> similar à humana exibida por sistemas de software, além de também ser um campo de estudo acadêmico. Os principais pesquisadores e livros didáticos definem o campo como "o estudo e projeto de agentes inteligentes", onde um <a href="/wiki/Agente_inteligente" title="Agente inteligente">agente inteligente</a> é um sistema que percebe seu ambiente e toma atitudes que maximizam suas chances de sucesso. <a href="/wiki/Andreas_Kaplan" title="Andreas Kaplan">Andreas Kaplan</a> e Michael Haenlein definem a inteligência artificial como “uma capacidade do sistema para interpretar corretamente dados externos, aprender a partir desses dados e utilizar essas aprendizagens para atingir objetivos e tarefas específicos através de adaptação flexível”.<sup class="reference" i

In [14]:
paragrafos[0].text

'Inteligência artificial (por vezes mencionada pela sigla em português IA ou pela sigla em inglês AI - artificial intelligence) é a inteligência similar à humana exibida por sistemas de software, além de também ser um campo de estudo acadêmico. Os principais pesquisadores e livros didáticos definem o campo como "o estudo e projeto de agentes inteligentes", onde um agente inteligente é um sistema que percebe seu ambiente e toma atitudes que maximizam suas chances de sucesso. Andreas Kaplan e Michael Haenlein definem a inteligência artificial como “uma capacidade do sistema para interpretar corretamente dados externos, aprender a partir desses dados e utilizar essas aprendizagens para atingir objetivos e tarefas específicos através de adaptação flexível”.[1] John McCarthy, quem cunhou o termo em 1956 ("numa conferência de especialistas celebrada em Darmouth Colege" Gubern, Román: O Eros Eletrónico), a define como "a ciência e engenharia de produzir máquinas inteligentes". É uma área de p

In [15]:
conteudo = ''
for p in paragrafos:
  conteudo += p.text

In [16]:
conteudo

'Inteligência artificial (por vezes mencionada pela sigla em português IA ou pela sigla em inglês AI - artificial intelligence) é a inteligência similar à humana exibida por sistemas de software, além de também ser um campo de estudo acadêmico. Os principais pesquisadores e livros didáticos definem o campo como "o estudo e projeto de agentes inteligentes", onde um agente inteligente é um sistema que percebe seu ambiente e toma atitudes que maximizam suas chances de sucesso. Andreas Kaplan e Michael Haenlein definem a inteligência artificial como “uma capacidade do sistema para interpretar corretamente dados externos, aprender a partir desses dados e utilizar essas aprendizagens para atingir objetivos e tarefas específicos através de adaptação flexível”.[1] John McCarthy, quem cunhou o termo em 1956 ("numa conferência de especialistas celebrada em Darmouth Colege" Gubern, Román: O Eros Eletrónico), a define como "a ciência e engenharia de produzir máquinas inteligentes". É uma área de p

**Pré-processamento da base de dados**

In [17]:
conteudo = conteudo.lower()
conteudo

'inteligência artificial (por vezes mencionada pela sigla em português ia ou pela sigla em inglês ai - artificial intelligence) é a inteligência similar à humana exibida por sistemas de software, além de também ser um campo de estudo acadêmico. os principais pesquisadores e livros didáticos definem o campo como "o estudo e projeto de agentes inteligentes", onde um agente inteligente é um sistema que percebe seu ambiente e toma atitudes que maximizam suas chances de sucesso. andreas kaplan e michael haenlein definem a inteligência artificial como “uma capacidade do sistema para interpretar corretamente dados externos, aprender a partir desses dados e utilizar essas aprendizagens para atingir objetivos e tarefas específicos através de adaptação flexível”.[1] john mccarthy, quem cunhou o termo em 1956 ("numa conferência de especialistas celebrada em darmouth colege" gubern, román: o eros eletrónico), a define como "a ciência e engenharia de produzir máquinas inteligentes". é uma área de p

In [18]:
lista_sentencas = nltk.sent_tokenize(conteudo)
#lista_sentencas

In [19]:
len(lista_sentencas)

229

In [20]:
type(lista_sentencas)

list

In [21]:
lista_sentencas[1]

'os principais pesquisadores e livros didáticos definem o campo como "o estudo e projeto de agentes inteligentes", onde um agente inteligente é um sistema que percebe seu ambiente e toma atitudes que maximizam suas chances de sucesso.'

In [22]:
# versao 3.0.5
pln = spacy.load('pt_core_news_sm')

In [23]:
stop_words = spacy.lang.pt.stop_words.STOP_WORDS

In [24]:
print(stop_words)

{'antes', 'fim', 'perto', 'está', 'novas', 'nós', 'deverá', 'ao', 'diante', 'aquele', 'lado', 'estiveram', 'dá', 'geral', 'ambas', 'sob', 'sua', 'quero', 'sabe', 'tudo', 'apontar', 'às', 'podem', 'nossa', 'cá', 'dezoito', 'fazia', 'porquanto', 'vindo', 'vos', 'quais', 'acerca', 'boa', 'porém', 'vão', 'fez', 'maioria', 'pouco', 'vossos', 'tentei', 'assim', 'talvez', 'segundo', 'usar', 'dizer', 'próxima', 'tive', 'vez', 'teu', 'estivestes', 'esteve', 'povo', 'cento', 'ela', 'pôde', 'muito', 'que', 'local', 'ponto', 'bastante', 'sexta', 'o', 'têm', 'pontos', 'ora', 'tanta', 'fará', 'seria', 'temos', 'das', 'até', 'fazeis', 'ele', 'tenho', 'quieta', 'tu', 'aquilo', 'do', 'número', 'à', 'põem', 'onze', 'tempo', 'doze', 'muitos', 'poderá', 'tentaram', 'possível', 'faz', 'tivemos', 'a', 'ambos', 'em', 'meu', 'contra', 'outros', 'mais', 'grandes', 'dois', 'apoio', 'outras', 'desde', 'próximo', 'somente', 'valor', 'teus', 'certeza', 'nessa', 'da', 'foi', 'cuja', 'ter', 'além', 'seis', 'sétima',

In [25]:
type(stop_words)

set

In [26]:
len(stop_words)

416

In [27]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [28]:
def preprocessamento(texto):
  # URLs
  texto = re.sub(r"https?://[A-Za-z0-9./]+", ' ', texto)

  # espacos em branco
  texto = re.sub(r" +", ' ', texto)

  documento = pln(texto)
  lista = []
  for token in documento:
    lista.append(token.lemma_)

  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in string.punctuation]
  lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])

  return lista

In [29]:
texto_teste = 'https://www.ufabc.edu.br ' + lista_sentencas[0]
texto_teste

'https://www.ufabc.edu.br inteligência artificial (por vezes mencionada pela sigla em português ia ou pela sigla em inglês ai - artificial intelligence) é a inteligência similar à humana exibida por sistemas de software, além de também ser um campo de estudo acadêmico.'

In [30]:
resultado = preprocessamento(texto_teste)
resultado

'  inteligência artificial vezar mencionar pelar sigla português pelar sigla inglês ai artificial intelligence inteligência similar humano exibir software campar estudar acadêmico'

In [31]:
lista_sentencas_preprocessada = []
for i in range(len(lista_sentencas)):
  lista_sentencas_preprocessada.append(preprocessamento(lista_sentencas[i]))

In [35]:
for _ in range(5):
  i = random.randint(0, len(lista_sentencas) - 1)
  print(lista_sentencas[i])
  print(lista_sentencas_preprocessada[i])
  print('-----')

esse especialista procura saber se os seres-humanos seriam capazes de construir máquinas que poderiam compreende-los, resolver seus problemas e dirigir suas vidas, além de buscar respostas sobre o que aconteceria se, algum dia, essas máquinas se tornassem mais inteligentes do que os próprios humanos que as criaram.
especialista procurar seres-humanos seriar capaz construir máquina compreende-los resolver problema dirigir suar vidar buscar respostar sobrar acontecer algum dia máquina tornar inteligente humano criar
-----
se a crítica de minsky fosse válida a maquina criada por rosenblatt e bernard widrow não estaria em uso ainda hoje, e o mark i perceptron não seria o fundador da neuro-computação.
crítico minsky válido maquinar criar rosenblatt bernard widrow hoje mark perceptron seriar fundador neuro-computação
-----
não vai conseguir jogar xadrez ou guiar um veículo autônomo, por exemplo, já que não consegue elaborar estratégias funcionais e sim apenas executar comandos com base em al

### **03. Frases de boas-vindas**

In [36]:
textos_boas_vindas_entrada = ('olá', 'opa', 'oi')
textos_boas_vindas_respostas = ('olá', 'opa', 'oi', 'bem-vindo', 'como você está?', 'tudo bem?')

In [37]:
'olá tudo bem'.split()

['olá', 'tudo', 'bem']

In [38]:
def responder_saudacao(texto):
  for palavra in texto.split():
    if palavra.lower() in textos_boas_vindas_entrada:
      return random.choice(textos_boas_vindas_respostas)  

In [43]:
responder_saudacao('olá tudo bem?')

'oi'

### **04. Representação TF-IDF (*Term Frequency* - *Inverse Document Frequency*)**

### *Bag of words*

- *Este é o primeiro documento*
- *Este documento é o segundo documento*

**- este é o primeiro segundo documento**
- 1 1 1 1 0 1
- 1 1 1 0 1 2

- Palavras muito frequentes "dominam" o documento e podem não representar tanta informação sobre o contexto
- Maiores "pesos" para documentos longos do que para documentos pequenos

### TF-IDF (*Term Frequency* - *Inverse Document Frequency*)

- Redimensionar a frequência das palavras pela frequência com que aparecem em todos os documentos
- **Term frequency (TF)**: frequência da palavra no documento atual - TF = (número de vezes que o termo **t** aparece no documento) / (número de termos no documento)
- **Inverse document frequency (IDF)**: quão rara é a palavra nos documentos - IDF = log(N/n), **N** é o número de documentos e **n** é o número de documentos que o termo **t** apareceu
- **TF-IDF**: importância de uma palavra para um documento em uma coleção ou corpus

- Considerando um documento com 100 palavras no qual a palavra cachorro aparece 5 vezes
- TF = 5 / 100 = 0.05
- Temos 100 documentos no total (N) e a palavra cachorro aparece aparece em 20 desses documentos (n)
- IDF = log(100 / 20) = 0.699
- TF-IDF = 0.05 * 0.699 = 0.035

- **Quanto maior o valor do peso, mais raro é o termo. Quanto menor o peso, mais comum é o termo.**

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [67]:
frases_teste = lista_sentencas_preprocessada[:3]
frases_teste

['inteligência artificial vezar mencionar pelar sigla português pelar sigla inglês ai artificial intelligence inteligência similar humano exibir software campar estudar acadêmico',
 'principal pesquisador livro didáticos definir campar comer estudar projeto agente inteligente agente inteligente perceber ambientar tomar atitude maximizar suar chance sucesso',
 'andreas kaplan michael haenlein definir inteligência artificial comer “ umar capacidade parir interpretar corretamente externo aprender dessar utilizar aprendizagem parir atingir objetivos tarefar específico adaptação flexível ”']

In [68]:
frases_teste.append(frases_teste[0])
frases_teste

['inteligência artificial vezar mencionar pelar sigla português pelar sigla inglês ai artificial intelligence inteligência similar humano exibir software campar estudar acadêmico',
 'principal pesquisador livro didáticos definir campar comer estudar projeto agente inteligente agente inteligente perceber ambientar tomar atitude maximizar suar chance sucesso',
 'andreas kaplan michael haenlein definir inteligência artificial comer “ umar capacidade parir interpretar corretamente externo aprender dessar utilizar aprendizagem parir atingir objetivos tarefar específico adaptação flexível ”',
 'inteligência artificial vezar mencionar pelar sigla português pelar sigla inglês ai artificial intelligence inteligência similar humano exibir software campar estudar acadêmico']

In [69]:
vetores_palavras = TfidfVectorizer()
palavras_vetorizadas = vetores_palavras.fit_transform(frases_teste)

In [70]:
type(palavras_vetorizadas)

scipy.sparse.csr.csr_matrix

In [71]:
palavras_vetorizadas

<4x54 sparse matrix of type '<class 'numpy.float64'>'
	with 77 stored elements in Compressed Sparse Row format>

In [72]:
print(vetores_palavras.get_feature_names())

['acadêmico', 'adaptação', 'agente', 'ai', 'ambientar', 'andreas', 'aprender', 'aprendizagem', 'artificial', 'atingir', 'atitude', 'campar', 'capacidade', 'chance', 'comer', 'corretamente', 'definir', 'dessar', 'didáticos', 'específico', 'estudar', 'exibir', 'externo', 'flexível', 'haenlein', 'humano', 'inglês', 'inteligente', 'inteligência', 'intelligence', 'interpretar', 'kaplan', 'livro', 'maximizar', 'mencionar', 'michael', 'objetivos', 'parir', 'pelar', 'perceber', 'pesquisador', 'português', 'principal', 'projeto', 'sigla', 'similar', 'software', 'suar', 'sucesso', 'tarefar', 'tomar', 'umar', 'utilizar', 'vezar']


In [73]:
len(vetores_palavras.get_feature_names())

54

In [74]:
print(vetores_palavras.vocabulary_)

{'inteligência': 28, 'artificial': 8, 'vezar': 53, 'mencionar': 34, 'pelar': 38, 'sigla': 44, 'português': 41, 'inglês': 26, 'ai': 3, 'intelligence': 29, 'similar': 45, 'humano': 25, 'exibir': 21, 'software': 46, 'campar': 11, 'estudar': 20, 'acadêmico': 0, 'principal': 42, 'pesquisador': 40, 'livro': 32, 'didáticos': 18, 'definir': 16, 'comer': 14, 'projeto': 43, 'agente': 2, 'inteligente': 27, 'perceber': 39, 'ambientar': 4, 'tomar': 50, 'atitude': 10, 'maximizar': 33, 'suar': 47, 'chance': 13, 'sucesso': 48, 'andreas': 5, 'kaplan': 31, 'michael': 35, 'haenlein': 24, 'umar': 51, 'capacidade': 12, 'parir': 37, 'interpretar': 30, 'corretamente': 15, 'externo': 22, 'aprender': 6, 'dessar': 17, 'utilizar': 52, 'aprendizagem': 7, 'atingir': 9, 'objetivos': 36, 'tarefar': 49, 'específico': 19, 'adaptação': 1, 'flexível': 23}


In [75]:
print(vetores_palavras.idf_)

[1.51082562 1.91629073 1.91629073 1.51082562 1.91629073 1.91629073
 1.91629073 1.91629073 1.22314355 1.91629073 1.91629073 1.22314355
 1.91629073 1.91629073 1.51082562 1.91629073 1.51082562 1.91629073
 1.91629073 1.91629073 1.22314355 1.51082562 1.91629073 1.91629073
 1.91629073 1.51082562 1.51082562 1.91629073 1.22314355 1.51082562
 1.91629073 1.91629073 1.91629073 1.91629073 1.51082562 1.91629073
 1.91629073 1.91629073 1.51082562 1.91629073 1.91629073 1.51082562
 1.91629073 1.91629073 1.51082562 1.51082562 1.51082562 1.91629073
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.51082562]


In [76]:
palavras_vetorizadas.todense()

matrix([[0.19781901, 0.        , 0.        , 0.19781901, 0.        ,
         0.        , 0.        , 0.        , 0.32030307, 0.        ,
         0.        , 0.16015154, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.16015154, 0.19781901, 0.        , 0.        , 0.        ,
         0.19781901, 0.19781901, 0.        , 0.32030307, 0.19781901,
         0.        , 0.        , 0.        , 0.        , 0.19781901,
         0.        , 0.        , 0.        , 0.39563802, 0.        ,
         0.        , 0.19781901, 0.        , 0.        , 0.39563802,
         0.19781901, 0.19781901, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.19781901],
        [0.        , 0.        , 0.41650395, 0.        , 0.20825197,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.20825197, 0.13292454, 0.        , 0.20825197, 0.16418825,
         0.        , 0.16418825, 0.        , 

In [77]:
palavras_vetorizadas.todense().shape

(4, 54)

### **05: Entendimento da similaridade por cosseno**

- Link: https://en.wikipedia.org/wiki/Cosine_similarity
- Cálculos passo a passo: https://janav.wordpress.com/2013/10/27/tf-idf-and-cosine-similarity/
- Temos duas matrizes: TF e IDF
- Cosine Similarity (d1, d2) =  Dot product(d1, d2) / ||d1|| * ||d2||

In [78]:
from sklearn.metrics.pairwise import cosine_similarity

In [79]:
palavras_vetorizadas[0].todense()

matrix([[0.19781901, 0.        , 0.        , 0.19781901, 0.        ,
         0.        , 0.        , 0.        , 0.32030307, 0.        ,
         0.        , 0.16015154, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.16015154, 0.19781901, 0.        , 0.        , 0.        ,
         0.19781901, 0.19781901, 0.        , 0.32030307, 0.19781901,
         0.        , 0.        , 0.        , 0.        , 0.19781901,
         0.        , 0.        , 0.        , 0.39563802, 0.        ,
         0.        , 0.19781901, 0.        , 0.        , 0.39563802,
         0.19781901, 0.19781901, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.19781901]])

In [80]:
cosine_similarity(palavras_vetorizadas[0], palavras_vetorizadas[1])

array([[0.04257614]])

In [81]:
cosine_similarity(palavras_vetorizadas[0], palavras_vetorizadas[3])

array([[1.]])

In [82]:
similaridade = cosine_similarity(palavras_vetorizadas[0], palavras_vetorizadas)
similaridade

array([[1.        , 0.04257614, 0.08168341, 1.        ]])

In [83]:
similaridade.argsort()

array([[1, 2, 0, 3]])

In [84]:
i = similaridade.argsort()[0][-2]
i

0

In [85]:
i = i.flatten()
i

array([0])

### **06: Função para o chatbot responder ao usuário**

In [86]:
def responder(texto_usuario):
  resposta_chatbot = ''
  lista_sentencas_preprocessada.append(texto_usuario)

  tfidf = TfidfVectorizer()
  palavras_vetorizadas = tfidf.fit_transform(lista_sentencas_preprocessada)

  similaridade = cosine_similarity(palavras_vetorizadas[-1], palavras_vetorizadas)

  indice_sentenca = similaridade.argsort()[0][-2]
  vetor_similar = similaridade.flatten()
  vetor_similar.sort()
  vetor_encontrado = vetor_similar[-2]

  if (vetor_encontrado == 0):
    resposta_chatbot = resposta_chatbot + 'Desculpe, mas não entendi!'
    return resposta_chatbot
  else:
    resposta_chatbot = resposta_chatbot + lista_sentencas[indice_sentenca]
    return resposta_chatbot

### **07. Conversação**

In [87]:
continuar = True
print('Olá, sou um chatbot e vou responder perguntas sobre inteligência artificial: ')
while (continuar == True):
  texto_usuario = input()
  texto_usuario = texto_usuario.lower()
  if (texto_usuario != 'sair'):
    if (responder_saudacao(texto_usuario) != None):
      print('Chatbot: ' + responder_saudacao(texto_usuario))
    else:
      print('Chatbot: ')
      print(responder(preprocessamento(texto_usuario)))
      lista_sentencas_preprocessada.remove(preprocessamento(texto_usuario))
  else:
    continuar = False
    print('Chatbot: Até breve!')

Olá, sou um chatbot e vou responder perguntas sobre inteligência artificial: 
inteligência
Chatbot: 
estes autores argumentam que toda inteligência apenas parece inteligência, sem necessariamente o ser.
inteligência artificial
Chatbot: 
outra definição de inteligência artificial é a inteligência que surge de um "dispositivo artificial".
alan turing
Chatbot: 
uma contribuição prática de alan turing foi o que se chamou depois de teste de turing (tt),[18] de 1950: em lugar de responder à pergunta "podem-se ter computadores inteligentes?"
ia fraca
Chatbot: 
alguns autores sustentam que se a ia fraca é possível, então também o é a forte.
ia forte
Chatbot: 
poucos têm sido os progressos na ia forte.
sigla
Chatbot: 
inteligência artificial (por vezes mencionada pela sigla em português ia ou pela sigla em inglês ai - artificial intelligence) é a inteligência similar à humana exibida por sistemas de software, além de também ser um campo de estudo acadêmico.
deep blue
Chatbot: 
grandes empresas 

**Referências**:

Notebook disponibilizado no Curso [Chatbots com Python e Dialogflow: O Guia para Iniciantes](https://iaexpert.academy/courses/chatbots-python-dialogflow-guia-iniciantes/).

**Mais informações**: [Create your chatbot using Python NLTK](https://medium.com/@ritidass29/create-your-chatbot-using-python-nltk-88809fa621d1).